In [1]:
from kubernetes import client, config
from kubernetes.client.rest import ApiException
import time


from kubernetes import client, config

In [9]:
#to create a deployment for a pod
def create_deployment(api_instance, name, namespace, container_image, container_port):
    # Configureate Pod template container
    container = client.V1Container(
        name=name,
        image=container_image,
        ports=[client.V1ContainerPort(container_port=container_port)])
    # Create and configurate a spec section
    template = client.V1PodTemplateSpec(
        metadata=client.V1ObjectMeta(labels={"app": name}),
        spec=client.V1PodSpec(containers=[container]))
    # Create the specification of deployment
    spec = client.V1DeploymentSpec(
        replicas=1,
        template=template,
        selector={'matchLabels': {'app': name}})
    # Instantiate the deployment object
    deployment = client.V1Deployment(
        api_version="apps/v1",
        kind="Deployment",
        metadata=client.V1ObjectMeta(name=name),
        spec=spec)
    # Create a deployment
    api_response = api_instance.create_namespaced_deployment(
        body=deployment,
        namespace=namespace)
    print("Deployment created. status='%s'" % str(api_response.status))

In [ ]:
#using prometheus to sum all the network_receive_bytes_total for a node in a given time




In [10]:
#using the above funtion to create a nginx pod in default namespace
def main():
    config.load_kube_config()
    apps_v1 = client.AppsV1Api()
    create_deployment(apps_v1, "nginx", "default", "nginx:1.7.9", 80)

if __name__ == '__main__':
    main()

Deployment created. status='{'available_replicas': None,
 'collision_count': None,
 'conditions': None,
 'observed_generation': None,
 'ready_replicas': None,
 'replicas': None,
 'unavailable_replicas': None,
 'updated_replicas': None}'


In [2]:

def get_pod_node(pod_name):
    # Load the kube config and Create a CoreV1Api instance
    config.load_kube_config()
    v1 = client.CoreV1Api()

    try:
        # Fetch the pods across all namespaces
        pods = v1.list_pod_for_all_namespaces().items

        # Filter the list for the specified pod name
        matching_pods = [pod for pod in pods if pod.metadata.name == pod_name]

        # Handle cases where no matches or multiple matches are found
        if not matching_pods:
            print(f"No pod found with name {pod_name}")
            return None
        elif len(matching_pods) > 1: # multiple pods with the same name in different namespaces
            print(f"Multiple pods found with the name {pod_name}. Specify a namespace.")
            return None
        else:
            print(f"Pod {pod_name} is running at node: {matching_pods[0].spec.node_name}")
            return matching_pods[0].spec.node_name

    except client.exceptions.ApiException as e:
        print(f"An error occurred: {e.reason}. {e.body}")
        return None



def change_node_for_pod(pod_name, pod_nameapce, new_assigned_node_name):
    # Beofre scheduling the pod:
    print("********************Before scheduling the pod ********************")
    
    # get node name
    old_assigned_node_name = get_pod_node(pod_name)
    # Load the kube config
    config.load_kube_config()
    # Create API instances
    core_v1 = client.CoreV1Api()

    try:
        # Step 1: Cordon all nodes except the new node
        # Get the list of nodes
        nodes = core_v1.list_node().items
        # Loop through the nodes
        for node in nodes:
            # Skip the new node
            if node.metadata.name == new_assigned_node_name:
                continue
            # Cordon the node
            body = client.V1Node(spec=client.V1NodeSpec(unschedulable=True))
            core_v1.patch_node(name=node.metadata.name, body=body)
            #print all cordoned nodes
            # print(f"Node {node.metadata.name} has been cordoned")
            

        # Get the existing pod details & specify the namespace (different namespace may have pods with same name)
        pod = core_v1.read_namespaced_pod(name=pod_name, namespace=pod_nameapce)
        
        # Check if the pod is on the old node
        if pod.spec.node_name != old_assigned_node_name:
            print(f"The pod is not on the node {old_assigned_node_name}")
            return
        # Set the node name in the pod spec to the new node
        pod.spec.node_name = new_assigned_node_name

        # Remove fields that should not be set when creating a new object
        pod.metadata.resource_version = None
        pod.metadata.uid = None
        pod.metadata.self_link = None
        pod.metadata.creation_timestamp = None
        pod.status = None

        # Delete the old pod
        core_v1.delete_namespaced_pod(name=pod_name, namespace=pod_nameapce)

        # Wait for the pod to be fully deleted
        while True:
            try:
                core_v1.read_namespaced_pod(name=pod_name, namespace=pod_nameapce)
                time.sleep(0) # change from 1 to 0
            except ApiException as e:
                if e.status == 404:
                    break

        # Step 2: Create a new pod on the new node
        core_v1.create_namespaced_pod(namespace=pod_nameapce, body=pod)
        print(f"Pod {pod_name} has been moved to node {new_assigned_node_name}")

    except ApiException as e:
        print(f"An error occurred: {e.reason}. {e.body}")
        return

    # After scheduling the pod:
    print("********************After scheduling the pod ********************")

    # Step 3: Uncordon the all cordoned nodes
    # Get the list of nodes
    nodes = core_v1.list_node().items
    # Loop through the nodes
    for node in nodes:
        # Skip the new node
        if node.metadata.name == new_assigned_node_name:
            continue
        # Uncordon the all cordoned nodes
        body = client.V1Node(spec=client.V1NodeSpec(unschedulable=False))
        core_v1.patch_node(name=node.metadata.name, body=body)
        #print all cordoned nodes
        # print(f"Node {node.metadata.name} has been uncordoned")
    
    
    # get the new name for the pod and print it
    new_pod_name = pod.metadata.name
    print(f"New pod name is: {new_pod_name}")
        


In [7]:
        
# change pod to another node        
change_node_for_pod(pod_name="nginx-thrift-86c98b895b-rpnhr", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-7")

********************Before scheduling the pod ********************
Pod nginx-thrift-86c98b895b-rpnhr is running at node: k8s-worker-3
Pod nginx-thrift-86c98b895b-rpnhr has been moved to node k8s-worker-7
********************After scheduling the pod ********************
New pod name is: nginx-thrift-86c98b895b-rpnhr


In [8]:
# change pod to another node        
change_node_for_pod(pod_name="user-timeline-service-65d5766796-f2b7h", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-8")



********************Before scheduling the pod ********************
Pod user-timeline-service-65d5766796-f2b7h is running at node: k8s-worker-3
Pod user-timeline-service-65d5766796-f2b7h has been moved to node k8s-worker-8
********************After scheduling the pod ********************
New pod name is: user-timeline-service-65d5766796-f2b7h


In [9]:
# change pod to another node        
change_node_for_pod(pod_name="post-storage-mongodb-5d7bfcd455-spqgt", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-9")



********************Before scheduling the pod ********************
Pod post-storage-mongodb-5d7bfcd455-spqgt is running at node: k8s-worker-1
Pod post-storage-mongodb-5d7bfcd455-spqgt has been moved to node k8s-worker-9
********************After scheduling the pod ********************
New pod name is: post-storage-mongodb-5d7bfcd455-spqgt


In [10]:
change_node_for_pod(pod_name="url-shorten-service-6d8bbf75f8-fw7dk", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-3")
change_node_for_pod(pod_name="user-service-74685b9465-zxc6q", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-2")
change_node_for_pod(pod_name="social-graph-redis-7d9b494f66-tnhk8", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-1")
change_node_for_pod(pod_name="unique-id-service-6b88dd4658-4bhpf", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-2")

change_node_for_pod(pod_name="media-frontend-75b5d868d7-f7rks", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-2")






********************Before scheduling the pod ********************
Pod url-shorten-service-6d8bbf75f8-fw7dk is running at node: k8s-worker-9
Pod url-shorten-service-6d8bbf75f8-fw7dk has been moved to node k8s-worker-3
********************After scheduling the pod ********************
New pod name is: url-shorten-service-6d8bbf75f8-fw7dk
********************Before scheduling the pod ********************
Pod user-service-74685b9465-zxc6q is running at node: k8s-worker-8
Pod user-service-74685b9465-zxc6q has been moved to node k8s-worker-2
********************After scheduling the pod ********************
New pod name is: user-service-74685b9465-zxc6q
********************Before scheduling the pod ********************
Pod social-graph-redis-7d9b494f66-tnhk8 is running at node: k8s-worker-7
Pod social-graph-redis-7d9b494f66-tnhk8 has been moved to node k8s-worker-1
********************After scheduling the pod ********************
New pod name is: social-graph-redis-7d9b494f66-tnhk8
**********

In [11]:
change_node_for_pod(pod_name="post-storage-memcached-675b5c57bd-lsskt", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-3")


********************Before scheduling the pod ********************
Pod post-storage-memcached-675b5c57bd-lsskt is running at node: k8s-worker-9
Pod post-storage-memcached-675b5c57bd-lsskt has been moved to node k8s-worker-3
********************After scheduling the pod ********************
New pod name is: post-storage-memcached-675b5c57bd-lsskt


In [12]:
change_node_for_pod(pod_name="jaeger-7455bdbc47-2zqsm", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-1")


********************Before scheduling the pod ********************
Pod jaeger-7455bdbc47-2zqsm is running at node: k8s-worker-7
Pod jaeger-7455bdbc47-2zqsm has been moved to node k8s-worker-1
********************After scheduling the pod ********************
New pod name is: jaeger-7455bdbc47-2zqsm


In [13]:
change_node_for_pod(pod_name="media-service-7848df64c4-gpqn8", pod_nameapce="social-network" , new_assigned_node_name="k8s-worker-2")


********************Before scheduling the pod ********************
Pod media-service-7848df64c4-gpqn8 is running at node: k8s-worker-8
Pod media-service-7848df64c4-gpqn8 has been moved to node k8s-worker-2
********************After scheduling the pod ********************
New pod name is: media-service-7848df64c4-gpqn8
